In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

In [ ]:
lx = 100.
ly = 100.
nlay = 1
nrow = 20
ncol = 20
delr = lx / ncol
delc = ly / nrow
top = 0
botm = -100
nper = 10 * 24
nstp = 1
perlen = 10.
steady = False
ibound = np.ones((nlay, nrow, ncol), dtype=np.int)
ibound[0, 0, 0] = -1
ibound[0, nrow - 1, ncol - 1] = -1
strt = np.zeros((nlay, nrow, ncol), dtype=np.float)
strt[0, 0, 0] = 1.

In [ ]:
#tidal information
def tide(a, b, c, d, t):
    return a + b * np.sin(c + d * t)

t = np.linspace(0, 10., 10*24 + 1)
stage = tide(0., 1, 0., 4. * np.pi, t)
plt.plot(t, stage, 'bo-')
plt.xlim(0, 10)

In [ ]:
chd_spd = {}
oc_spd = {}
for kper in range(nper):
    chd_spd[kper] = [[0, 0, 0, stage[kper], stage[kper+1]]]
    oc_spd[(kper, 0)] = ['save head', 'save budget']

In [ ]:
model_ws = './work/exFlopyIntroB'
name = 'fpintro'
m = flopy.modflow.Modflow(modelname=name, model_ws=model_ws, 
                         exe_name=config.mfexe)
dis = flopy.modflow.ModflowDis(m, nlay, nrow, ncol, nper,
                              delr, delc, top=top, botm=botm,
                              perlen=perlen, steady=steady)
bas = flopy.modflow.ModflowBas(m, ibound=ibound, strt=strt)
chd = flopy.modflow.ModflowChd(m, stress_period_data=chd_spd)
lpf = flopy.modflow.ModflowLpf(m, ipakcb=52)
pcg = flopy.modflow.ModflowPcg(m)
oc = flopy.modflow.ModflowOc(m, stress_period_data=oc_spd)
m.write_input()
m.run_model()

In [ ]:
fname = model_ws + '/' + name + '.hds'
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_alldata()

fname = model_ws + '/' + name + '.cbc'
budobj = flopy.utils.CellBudgetFile(fname)
frf = budobj.get_data(text='FLOW RIGHT FACE')
flf = budobj.get_data(text='FLOW FRONT FACE')
head.shape

In [ ]:
kper = 0
mm = flopy.plot.ModelMap(model=m)
mm.plot_grid()
mm.plot_ibound()
mm.plot_array(head[kper], cmap='jet')
mm.contour_array(head[0])
mm.plot_discharge(frf[kper], frf[kper])
plt.title('Stress Period {}'.format(kper+1))

In [ ]:
icells = [(0, 0, 0), (0, int(nrow/2), int(ncol/2)), (0, nrow-1, ncol-1)]
ts = hdobj.get_ts(icells)
plt.plot(ts[:, 0], ts[:, 1:])

In [ ]:
import time
from IPython.display import clear_output, display
animate = False
if animate:
    f = plt.figure(figsize=(10, 5))
    ax = plt.subplot(1, 1, 1)
    for kper in range(24):
        mm = flopy.plot.ModelMap(model=m)
        mm.plot_grid()
        mm.plot_ibound()
        mm.plot_array(head[kper], vmin=-1, vmax=1, cmap='jet')
        mm.contour_array(head[0])
        mm.plot_discharge(frf[kper], frf[kper])
        plt.title('Stress Period {}'.format(kper+1))        
        time.sleep(0.01)
        clear_output(True)
        display(f)
        plt.gca().cla()
    print('Done.');